In [33]:
import wandb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


In [60]:
wandb.login(key="8a88a8c49d1c2d31b8677fe0b8eb7d3e3a031f83")
api = wandb.Api()
fill_value = "N/A"

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /export/home/0schindl/.netrc


In [61]:
sweep_id = "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_selection_method/wk4w5q0t"
sweep = api.sweep(sweep_id)
runs = sweep.runs
cil_datasets = ["cifar100", "imagenetr", "cub", "imageneta", "vtab", "cars", "omnibenchmark"]
dil_datasets = ["cddb", "dil_imagenetr", "limited_domainnet"]


In [62]:
data = []
# whats with cddb ???
for run in runs:
    config = run.config
    summary = run.summary
    

    dataset = config.get("dataset")
    selection_method = config.get("selection_method")
    run_id = run.id
    state = run.state
    num_E = config.get("moe_max_experts")




    if state == "finished" and dataset in cil_datasets and (selection_method == "around" or selection_method == "inv_ws_div"):
        # Get the history of the run
        history = run.history()

        run_data = {
            "run_id": run_id,
            "dataset": dataset,
            "selection_method": selection_method           
            }
        

        # End accuracy of tasks
        for key, value in summary.items():
            if key.startswith("task_") and key.endswith("/acc") and key != "task_mean/acc" and key != "task_wmean/acc":
                t = int(key.split("_")[1].split("/")[0])

                # is an initial task
                if num_E <= t:
                    continue

                key = "end_" + key
                run_data[key] = value
           
        



        # Initial accuracy of tasks
        for line in history.items():
            key = line[0]
            if key.startswith("task_") and key.endswith("/acc") and key != "task_mean/acc" and key != "task_wmean/acc":
                t = int(key.split("_")[1].split("/")[0])

                # is an initial task
                if num_E <= t:
                    continue

                values = line[1]
                filtered_values = values[~np.isnan(values)]
                index = num_E - t - 1
                base_acc = filtered_values.iloc[index]
                key = "initial_" + key
                run_data[key] = base_acc
        



        data.append(run_data)
        
        
df_sweep = pd.DataFrame(data)
print("\nDataFrame der Sweep-Runs:")
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', None)
print(df_sweep)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)



DataFrame der Sweep-Runs:
      run_id        dataset selection_method  end_task_0/acc  end_task_1/acc  end_task_2/acc  end_task_3/acc  end_task_4/acc  initial_task_2/acc  initial_task_4/acc  initial_task_3/acc  initial_task_0/acc  initial_task_1/acc
0   53rkgrjf  omnibenchmark       inv_ws_div        0.723618        0.553512        0.424749        0.715719        0.600000            0.596990            0.676667            0.744147            0.735343            0.607023
1   da82toeh           cars       inv_ws_div        0.341829        0.318859        0.207776        0.229858        0.261239            0.317132            0.349939            0.368483            0.422789            0.414392
2   as2k8u4j           vtab       inv_ws_div        0.786524        0.772707        0.883538             NaN             NaN            0.901051                 NaN                 NaN            0.818010            0.810443
3   923atyey           vtab           around        0.771411        0.780

In [63]:
df_forgetting = df_sweep.copy()

# Identify end and initial task accuracy columns
end_cols = [col for col in df_sweep.columns if col.startswith('end_task_') and col.endswith('/acc')]
initial_cols = [col for col in df_sweep.columns if col.startswith('initial_task_') and col.endswith('/acc')]

# Calculate forgetting for each task
forgetting_values = []
for index, row in df_sweep.iterrows():
    row_forgetting = []
    for i in range(5):  # Assuming there are tasks 0 to 4
        end_col = f'end_task_{i}/acc'
        initial_col = f'initial_task_{i}/acc'
        if end_col in row and initial_col in row and pd.notna(row[end_col]) and pd.notna(row[initial_col]):
            forgetting = row[initial_col] - row[end_col]
            row_forgetting.append(forgetting)
    # Calculate the average forgetting for the row, ignoring NaNs
    avg_forgetting = np.nanmean(row_forgetting)
    forgetting_values.append(avg_forgetting)

# Add the average forgetting as a new column
df_forgetting['average_forgetting'] = forgetting_values

print("\nDataFrame der Sweep-Runs:")
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', None)
print(df_forgetting.sort_values(by='dataset'))
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


DataFrame der Sweep-Runs:
      run_id        dataset selection_method  end_task_0/acc  end_task_1/acc  end_task_2/acc  end_task_3/acc  end_task_4/acc  initial_task_2/acc  initial_task_4/acc  initial_task_3/acc  initial_task_0/acc  initial_task_1/acc  average_forgetting
1   da82toeh           cars       inv_ws_div        0.341829        0.318859        0.207776        0.229858        0.261239            0.317132            0.349939            0.368483            0.422789            0.414392            0.102635
5   kjh5dj04           cars           around        0.361319        0.282878        0.211422        0.225118        0.250304            0.339004            0.349939            0.366114            0.458771            0.403226            0.117202
12  q9r1xc9a       cifar100       inv_ws_div        0.763000        0.777000        0.644000        0.811000        0.858000            0.721000            0.880000            0.851000            0.811000            0.810000            0.